From .mat files we will sample data on minute interval for the race

In [1]:
import scipy.io as si
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import plotly.express as px

from datetime import timedelta

from utils import * # get_dict_len, get_minute_data, get_master_table

1. Read all files in mat data folder

In [87]:
dataDir = 'matdata/' #file is save 
mats = []
for file in os.listdir(dataDir) :
    mats.append(si.loadmat(dataDir+file))

C:\Users\nguyeneq\Anaconda3\envs\sb\lib\site-packages\scipy\io\matlab\mio.py:224: MatReadWarning:

Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files



Store each .mat file as a dictionary for the day

In [88]:
prep_08 = {k:v for k, v in mats[0].items() if k[0] != '_'}
#dict_len_08 = get_dict_len(prep_08)
prep_09 = {k:v for k, v in mats[1].items() if k[0] != '_'}
#dict_len_09 = get_dict_len(prep_09)
prep_10 = {k:v for k, v in mats[2].items() if k[0] != '_'}
#dict_len_10 = get_dict_len(prep_10)
prep_11 = {k:v for k, v in mats[3].items() if k[0] != '_'}
#dict_len_11 = get_dict_len(prep_11)
prep_12 = {k:v for k, v in mats[4].items() if k[0] != '_'}
#dict_len_12 = get_dict_len(prep_12)
prep_13 = {k:v for k, v in mats[5].items() if k[0] != '_'}
#dict_len_13 = get_dict_len(prep_13)

# create a prep list
prep_list = [prep_08, prep_09, prep_10, prep_11, prep_12, prep_13]

In [90]:
# There is one problem with column date and time in .mat file
# eg. .mat file for 13/10 but the day in the datetime column of 
# this file still indicates 08/10

def add_day(time_string, count):
    # count is the number of the day
    # if the first day then count = 0
    # second day then count = 1
    time_string = time_string[0]
    timeformat = '%d-%m-%Y %H:%M:%S.%f'
    time_point = datetime.datetime.strptime(time_string, timeformat)
    time_point = time_point + timedelta(days = count)
    return time_point.strftime("%d-%m-%Y %H:%M:%S.%f")

In [121]:
prep_dict = {}
prep_dict['watt'] = [13, 1329]
prep_dict['batt'] = [4, 1328]
prep_dict['odo'] = [5, 1038]
prep_dict['volt'] = [6, 1784]
prep_dict['power'] = [11, 1299]
prep_dict['motor'] = [15, 1026]
prep_dict['speed'] = [16, 1027]

In [131]:
def get_master_table(prep_list, prep_dict, name):
    # prep_dict_ele from prep_dict
    count = 0
    for prep_ele in prep_list:
        print('Fetching data from {} .mat file of day {}/6, {} rows, packet ID {}...'.format(name,count+1,len(prep_ele[list(prep_ele.keys())[0]]), prep_dict[name][1] ))
        dict_len = get_dict_len(prep_ele)
        # get datetime table from the .mat file
        t_datetime = make_table(dict_len[list(dict_len.keys())[0]],prep_ele)
        # get table of interest fromt .mat file, for example watt, batt, odo
        t_interest = make_table(dict_len[list(dict_len.keys())[prep_dict[name][0]]],prep_ele) # 341822 	
        
        # concatenate the time table, and the table of interest
        interest_time = t_datetime[t_datetime['ID'] == prep_dict[name][1]]
        interest_time = interest_time.reset_index()
        interest_time = interest_time.drop(['index'], axis=1)
        interest_time = pd.concat([interest_time,t_interest],axis=1)

        if count == 0:
            interest_time['dateAndTimeData'] = interest_time.apply(lambda x: x.dateAndTimeData[0], axis = 1)
            # interest time master is the table that contains data of all days
            interest_time_master = interest_time
        else:
            interest_time['dateAndTimeData'] = interest_time.apply(lambda x: add_day(x.dateAndTimeData,count), axis = 1)
            interest_time_master = interest_time_master.append(interest_time,ignore_index=True)
        count += 1
    # save to dist
    interest_time_master.to_csv('simple/' + name + '_time_master_trial.csv')#, index = False)
    # change into minute interval
    interest_time_master['dateAndTimeData'] = interest_time_master.apply(lambda x: get_minute_data(x.dateAndTimeData), axis = 1)
    # drop duplicated values and keep the first row only
    interest_time_master = interest_time_master.drop_duplicates(subset=['dateAndTimeData'], keep = 'first').reset_index().drop(['index'],axis = 1)
    
    if name == 'odo':
        # with odometer table, we need to process the total sum of distance at each time point because
        # it was set at zero again after break. 
        c = 0.
        interest_time_master['distkm'] = 0.
        for idx, loc in interest_time_master.iterrows():


            if idx < len(interest_time_master) - 1:
                if interest_time_master.at[idx+1,'odometerkm'] < interest_time_master.at[idx,'odometerkm']:
                    c+=interest_time_master.at[idx,'odometerkm']
                interest_time_master.at[idx,'distkm'] = interest_time_master.at[idx,'odometerkm'] + c
            else:
                c+=interest_time_master.at[idx,'odometerkm']
                interest_time_master.at[idx,'distkm'] = interest_time_master.at[idx,'odometerkm'] + c
    return interest_time_master

We will make variable of interests (battery, wayy, motor, speed, volt, time, power and odo

In [132]:
watt_time_master = get_master_table(prep_list, prep_dict, 'watt')
batt_time_master = get_master_table(prep_list, prep_dict, 'batt')
motor_time_master = get_master_table(prep_list, prep_dict, 'motor')
speed_time_master = get_master_table(prep_list, prep_dict, 'speed')
volt_time_master = get_master_table(prep_list, prep_dict, 'volt')
power_time_master = get_master_table(prep_list, prep_dict, 'power')
odo_time_master = get_master_table(prep_list, prep_dict,  'odo')

Fetch data from watt .mat file, 2305665 rows, packet ID 1329
Fetch data from watt .mat file, 2795559 rows, packet ID 1329
Fetch data from watt .mat file, 2698812 rows, packet ID 1329
Fetch data from watt .mat file, 2395562 rows, packet ID 1329
Fetch data from watt .mat file, 2943606 rows, packet ID 1329
Fetch data from watt .mat file, 2033613 rows, packet ID 1329
Fetch data from batt .mat file, 2305665 rows, packet ID 1328
Fetch data from batt .mat file, 2795559 rows, packet ID 1328
Fetch data from batt .mat file, 2698812 rows, packet ID 1328
Fetch data from batt .mat file, 2395562 rows, packet ID 1328
Fetch data from batt .mat file, 2943606 rows, packet ID 1328
Fetch data from batt .mat file, 2033613 rows, packet ID 1328
Fetch data from motor .mat file, 2305665 rows, packet ID 1026
Fetch data from motor .mat file, 2795559 rows, packet ID 1026
Fetch data from motor .mat file, 2698812 rows, packet ID 1026
Fetch data from motor .mat file, 2395562 rows, packet ID 1026
Fetch data from moto

In [151]:
temp = watt_time_master
temp =  pd.merge(temp, batt_time_master, how="left", on="dateAndTimeData")
temp =  pd.merge(temp, motor_time_master, how="left", on="dateAndTimeData")
temp =  pd.merge(temp, speed_time_master, how="left", on="dateAndTimeData")
temp =  pd.merge(temp, power_time_master, how="left", on="dateAndTimeData")
temp =  pd.merge(temp, odo_time_master, how="left", on="dateAndTimeData")
#temp =  pd.merge(temp, batt_time_master, how="left", on="dateAndTimeData")
temp = temp.dropna()

drop_cols = ['ID_x', 
        'dateAndTimeSerial_x', 
        'hvstatus', 
        'x531time', 
        'ID_y', 
        'dateAndTimeSerial_y', 
        'x530time', 'ID_x',
        'dateAndTimeSerial_x',  
        'x402time', 
        'ID_y', 
        'dateAndTimeSerial_y',
        'x403time', 
        'ID_x',
        'dateAndTimeSerial_x', 
        'esc_average_power', 
        'x513time', 
        'ID_y',
        'dateAndTimeSerial_y', 
        'busamphrs', 
        'odometer', 
        'odometerkm',
        'x40Etime']

temp = temp.drop(columns = drop_cols)

In [152]:
# read route file from WSCRoute2019 provided by WSU team, let's call route r
r = pd.read_csv('RouteWSC2019v2.csv', header = 0)
r.columns = ['dist', 'lat', 'long', 'bearing', 'elevation','speed_lim']
r.loc[r['speed_lim'] == 0,'speed_lim'] = 60
r['distkmitint'] = r.apply(lambda x: int(x.dist),axis = 1)
# sample by the km only
r = r.drop_duplicates(subset=['distkmitint'], keep = 'first').reset_index().drop(['index'],axis = 1)
# prepare temporary table
temp['distkmitint'] = temp.apply(lambda x: int(x.distkm),axis = 1)
# filter columns needed to join with temporary 
cols = ['bearing', 'elevation', 'speed_lim', 'distkmitint']
r = r.loc[:,cols]
# merge two tables
temp =  pd.merge(temp,r, how="left", on="distkmitint")
# drop nan value
temp = temp.dropna()

In [166]:
temp.head()

dateAndTimeData  watthrs  batterycurrent  batterypower  batteryvoltage  \
0  08-10-2017 08:31  42.3260        0.004229      0.648861         153.414   
1  08-10-2017 08:32  40.4781      -12.492607  -1887.495517         151.089   
2  08-10-2017 08:33  36.4669        1.198409    184.727551         154.144   
3  08-10-2017 08:34  43.7221       -0.826644   -127.113884         153.771   
4  08-10-2017 08:35  14.8229       -1.632047   -249.401326         152.815   

   motorcurrent   motorpower  motorpowerfilter  motorvoltage  motorvelocity  \
0      1.805073   275.100349        584.867934       152.404       202.9500   
1     15.216189  2278.898252        514.848534       149.768       322.0650   
2      1.267594   193.194017        -77.600631       152.410       700.0390   
3      3.395301   517.956579       1043.409429       152.551        83.7387   
4      4.615849   697.897908        692.142091       151.196       761.2000   

   vehiclespeed  vehiclevelocity    distkm  distkmitint     bearing  \
0     21.308138         5.918927  0.151248            0  225.764131   
1     33.814256         9.392849  0.206851            0  225.764131   
2     73.498494        20.416248  1.130570            1  264.423045   
3      8.791891         2.442192  1.784790            1  264.423045   
4     79.919941        22.199984  2.923380            2  251.649687   

   elevation  speed_lim  
0  36.211710       60.0  
1  36.211710       60.0  
2   8.496105       60.0  
3   8.496105       60.0  
4   9.000000       80.0

In [2]:
import pandas as pd

In [3]:
his = pd.read_csv('historical_sample.csv')

In [5]:
his.head()

dateAndTimeData  watthrs  batterycurrent  batterypower  batteryvoltage  \
0  08-10-2017 08:31  42.3260        0.004229      0.648861         153.414   
1  08-10-2017 08:32  40.4781      -12.492607  -1887.495517         151.089   
2  08-10-2017 08:33  36.4669        1.198409    184.727551         154.144   
3  08-10-2017 08:34  43.7221       -0.826644   -127.113884         153.771   
4  08-10-2017 08:35  14.8229       -1.632047   -249.401326         152.815   

   motorcurrent   motorpower  motorpowerfilter  motorvoltage  motorvelocity  \
0      1.805073   275.100349        584.867934       152.404       202.9500   
1     15.216189  2278.898252        514.848534       149.768       322.0650   
2      1.267594   193.194017        -77.600631       152.410       700.0390   
3      3.395301   517.956579       1043.409429       152.551        83.7387   
4      4.615849   697.897908        692.142091       151.196       761.2000   

   vehiclespeed  vehiclevelocity    distkm  distkmitint     bearing  \
0     21.308138         5.918927  0.151248            0  225.764131   
1     33.814256         9.392849  0.206851            0  225.764131   
2     73.498494        20.416248  1.130570            1  264.423045   
3      8.791891         2.442192  1.784790            1  264.423045   
4     79.919941        22.199984  2.923380            2  251.649687   

   elevation  speed_lim  
0  36.211710       60.0  
1  36.211710       60.0  
2   8.496105       60.0  
3   8.496105       60.0  
4   9.000000       80.0